<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/main/Copy_of_Mercury_Graphene_Sheet_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install dependencies and compile LAMMPS
%%capture
!apt-get update
!apt-get install -y build-essential cmake libfftw3-dev libjpeg-dev libpng-dev \
                    libopenmpi-dev openmpi-bin python3-dev python3-numpy git
# Clone the LAMMPS repository
%cd /content
!git clone -b stable https://github.com/lammps/lammps.git
%cd lammps

# Create a build directory and compile LAMMPS with required packages
!mkdir build
%cd build
!cmake ../cmake -DBUILD_SHARED_LIBS=yes \
                -DLAMMPS_EXCEPTIONS=yes \
                -DPKG_MOLECULE=yes \
                -DPKG_KSPACE=yes \
                -DPKG_RIGID=yes \
                -DPKG_MANYBODY=yes \
                -DPKG_USER-MISC=yes \
                -DPKG_PYTHON=yes \
                -DPYTHON_EXECUTABLE=`which python3`
!make -j4
!make install-python

# Return to the working directory
%cd /content/

In [7]:
import os
os.environ["OMP_NUM_THREADS"] = "4"

In [40]:
import os
import numpy as np
from scipy.spatial import cKDTree

def generate_graphene_layer(a=2.46, num_cells_x=10, num_cells_y=10, z=0.0, pore_radius=0.0):
    """
    Generate positions and bonds for a graphene layer with sp² hybridization.
    All carbons in this function will have the same molecule_id (1).
    """
    positions = []
    for i in range(num_cells_x):
        for j in range(num_cells_y):
            pos1 = (i * a * 1.5, j * a * np.sqrt(3), z)
            pos2 = (i * a * 1.5 + a * 0.75, j * a * np.sqrt(3) + a * (np.sqrt(3)/2), z)
            positions.append(pos1)
            positions.append(pos2)

    positions = np.array(positions)

    x_center = (num_cells_x * a * 1.5) / 2.0
    y_center = (num_cells_y * a * np.sqrt(3)) / 2.0

    dx = positions[:, 0] - x_center
    dy = positions[:, 1] - y_center
    distances = np.sqrt(dx**2 + dy**2)

    # No pore removal if pore_radius=0
    mask = distances >= pore_radius
    positions = positions[mask]

    num_atoms_layer = len(positions)

    # Assign C-C bonds ensuring three neighbors per carbon
    bonds = []
    bond_id = 1
    positions_2D = positions[:, :2]
    tree = cKDTree(positions_2D)
    cutoff_distance = 1.5 * a
    bond_counts = [0]*num_atoms_layer
    potential_pairs = list(tree.query_pairs(r=cutoff_distance))

    # Sort by distance so that closer neighbors get bonded first
    potential_pairs_sorted = sorted(potential_pairs, key=lambda pair: np.linalg.norm(positions_2D[pair[0]] - positions_2D[pair[1]]))

    for idx1, idx2 in potential_pairs_sorted:
        if bond_counts[idx1] < 3 and bond_counts[idx2] < 3:
            # Bond type 1 for C-C
            bonds.append((bond_id, 1, idx1 + 1, idx2 + 1))
            bond_id += 1
            bond_counts[idx1] += 1
            bond_counts[idx2] += 1

    angles = []
    return positions.tolist(), bonds, angles


def create_activated_carbon():
    """
    Create a 3-layer stack of graphene with no pore. Keep all carbon in molecule_id=1.
    """
    carbon_positions = []
    carbon_bonds = []
    carbon_angles = []
    layer_spacing = 3.35
    total_atoms = 0

    for layer in range(3):
        z = layer * layer_spacing
        layer_positions, layer_bonds, layer_angles = generate_graphene_layer(z=z, pore_radius=0.0)

        atom_offset = total_atoms
        for bond in layer_bonds:
            bond_id, bond_type, atom1, atom2 = bond
            # Renumber bonds and shift atom indices by atom_offset
            carbon_bonds.append((bond_id + len(carbon_bonds), bond_type, atom1 + atom_offset, atom2 + atom_offset))

        total_atoms += len(layer_positions)
        carbon_positions.extend(layer_positions)

    return carbon_positions, carbon_bonds, carbon_angles


def shift_positions(positions, shift_vector):
    return (np.array(positions) + np.array(shift_vector)).tolist()

def generate_random_position_box(box_bounds, existing_positions, min_distance=2.5, tolerance=0.1):
    max_attempts = 1000
    xlo, xhi, ylo, yhi, zlo, zhi = box_bounds
    for _ in range(max_attempts):
        x = np.random.uniform(xlo, xhi)
        y = np.random.uniform(ylo, yhi)
        z = np.random.uniform(zlo, zhi)
        pos = np.array([x, y, z])

        if existing_positions.size == 0:
            return pos
        distances = np.linalg.norm(existing_positions - pos, axis=1)
        if np.all(distances >= (min_distance - tolerance)):
            return pos
    return None

def place_sorbate(pos, orientation, bond_length=0.9572, bond_angle=104.52):
    theta, phi = orientation
    angle_rad = np.deg2rad(bond_angle / 2)
    x_offset = bond_length * np.sin(angle_rad) * np.cos(phi)
    y_offset = bond_length * np.sin(angle_rad) * np.sin(phi)
    z_offset = bond_length * np.cos(angle_rad)
    H1 = pos + np.array([x_offset, y_offset, z_offset])
    H2 = pos + np.array([-x_offset, -y_offset, z_offset])
    return H1, H2

def create_water_molecules(num_water, box_bounds, existing_positions, min_distance_O=2.5, min_distance_H=1.5):
    water_positions = []
    bond_length = 0.9572
    bond_angle = 104.52
    xlo, xhi, ylo, yhi, zlo, zhi = box_bounds

    for i in range(num_water):
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2*np.pi)
        orientation = (theta, phi)

        O = generate_random_position_box(box_bounds, existing_positions, min_distance=min_distance_O)
        if O is None:
            continue
        H1, H2 = place_sorbate(O, orientation, bond_length, bond_angle)

        if not (xlo <= H1[0] <= xhi and ylo <= H1[1] <= yhi and zlo <= H1[2] <= zhi and
                xlo <= H2[0] <= xhi and ylo <= H2[1] <= yhi and zlo <= H2[2] <= zhi):
            continue

        if existing_positions.size > 0:
            distances_O = np.linalg.norm(existing_positions - O, axis=1)
            distances_H1 = np.linalg.norm(existing_positions - H1, axis=1)
            distances_H2 = np.linalg.norm(existing_positions - H2, axis=1)
        else:
            distances_O = distances_H1 = distances_H2 = np.array([])

        if ((existing_positions.size == 0) or
            (np.all(distances_O >= min_distance_O) and
             np.all(distances_H1 >= min_distance_H) and
             np.all(distances_H2 >= min_distance_H))):
            water_positions.append((O, H1, H2))
            existing_positions = np.vstack([existing_positions, O, H1, H2])

    return water_positions, existing_positions

def add_cations(num_Mg, num_Zn, num_Ca, box_bounds, existing_positions):
    """
    Add Mg2+, Zn2+, Ca2+ ions. Return actually placed cations.
    """
    xlo, xhi, ylo, yhi, zlo, zhi = box_bounds
    cations = {'Mg': (6, 2.0, num_Mg),
               'Zn': (7, 2.0, num_Zn),
               'Ca': (8, 2.0, num_Ca)}
    added_cations = []

    for ion, (type_id, charge, ion_count) in cations.items():
        placed_ions = 0
        for _ in range(ion_count):
            pos = generate_random_position_box(box_bounds, existing_positions, min_distance=2.5)
            if pos is not None:
                added_cations.append((type_id, charge, pos))
                existing_positions = np.vstack([existing_positions, pos])
                placed_ions += 1
    return added_cations, existing_positions

def calculate_required_cl(total_cations):
    return int(round(total_cations))

def main():
    # Set random seed for reproducibility
    np.random.seed(42)

    # Box dimensions
    xlo, xhi = 0.0, 100.0
    ylo, yhi = 0.0, 100.0
    zlo, zhi = 0.0, 35.0
    box_bounds = (xlo, xhi, ylo, yhi, zlo, zhi)

    # Create activated carbon
    carbon_positions, carbon_bonds, carbon_angles = create_activated_carbon()
    num_cells_x = 10
    num_cells_y = 10
    shift_vector = (
        50.0 - (num_cells_x * 2.46 * 1.5) / 2.0,
        50.0 - (num_cells_y * 2.46 * np.sqrt(3)) / 2.0,
        17.5 - (3 * 3.35) / 2.0
    )
    carbon_positions = np.array(shift_positions(carbon_positions, shift_vector))
    num_c = len(carbon_positions)
    existing_positions = carbon_positions.copy()

    # Desired ions
    num_Hg = 50      # Hg2+
    num_Mg = 10      # Mg2+
    num_Zn = 10      # Zn2+
    num_Ca = 10      # Ca2+
    num_C_groups = 10
    num_carbonyl = 4
    num_hydroxyl = num_C_groups - num_carbonyl

    # Atom types and charges
    atom_types = {
        'C': 1,
        'O': 2,
        'H': 3,
        'Hg': 4,
        'Cl': 5,
        'Mg': 6,
        'Zn': 7,
        'Ca': 8
    }

    charges = {
        1: 0.0,    # C
        2: -0.834, # O (default for water), adjusted later for functional groups
        3: 0.417,  # H
        4: 2.0,    # Hg2+
        5: -1.0,   # Cl-
        6: 2.0,    # Mg2+
        7: 2.0,    # Zn2+
        8: 2.0     # Ca2+
    }

    # Build initial atom list
    atoms = []
    atom_id = 1
    molecule_id = 1
    for pos in carbon_positions:
        atoms.append([atom_id, molecule_id, atom_types['C'], charges[atom_types['C']], pos[0], pos[1], pos[2]])
        atom_id += 1

    # Identify edge carbons for functionalization
    bond_lengths = []
    for bond in carbon_bonds:
        _, _, a1, a2 = bond
        p1 = carbon_positions[a1 - 1]
        p2 = carbon_positions[a2 - 1]
        bl = np.linalg.norm(p1 - p2)
        bond_lengths.append(bl)
    avg_bond_length = np.mean(bond_lengths)

    x_center = 50.0
    y_center = 50.0
    distances = np.sqrt((carbon_positions[:,0] - x_center)**2 + (carbon_positions[:,1] - y_center)**2)
    max_distance = np.max(distances)
    edge_threshold = max_distance - (1.5 * avg_bond_length)

    edge_carbon_ids = [i+1 for i, d in enumerate(distances) if d >= edge_threshold]
    if len(edge_carbon_ids) < num_C_groups:
        additional_ids = [i+1 for i in range(num_c) if (i+1) not in edge_carbon_ids]
        edge_carbon_ids.extend(additional_ids[:num_C_groups - len(edge_carbon_ids)])
    functional_group_carbon_ids = edge_carbon_ids[:num_C_groups]

    # Add functional groups
    # Carbonyl: O = -0.834, add +0.834 to C
    # Hydroxyl: O = -0.417, H = +0.417, net zero
    for idx, c_id in enumerate(functional_group_carbon_ids):
        c_pos = carbon_positions[c_id - 1]
        direction = np.array([c_pos[0] - x_center, c_pos[1] - y_center, 0.0])
        norm = np.linalg.norm(direction)
        if norm == 0:
            direction = np.array([1.0, 0.0, 0.0])
        else:
            direction = direction / norm

        if idx < num_carbonyl:
            # Carbonyl
            O_pos = c_pos + direction * 1.2
            atoms.append([atom_id, molecule_id, atom_types['O'], -0.834, O_pos[0], O_pos[1], O_pos[2]])
            O_id = atom_id
            atom_id += 1
            # Add bond
            carbon_bonds.append((len(carbon_bonds) + 1, 3, c_id, O_id))
            # Adjust C charge
            atoms[c_id - 1][3] += 0.834
        else:
            # Hydroxyl
            O_pos = c_pos + direction * 1.2
            atoms.append([atom_id, molecule_id, atom_types['O'], -0.417, O_pos[0], O_pos[1], O_pos[2]])
            O_id = atom_id
            atom_id += 1
            carbon_bonds.append((len(carbon_bonds) + 1, 3, c_id, O_id))

            H_pos = O_pos + np.array([0.96, 0.0, 0.0])
            atoms.append([atom_id, molecule_id, atom_types['H'], 0.417, H_pos[0], H_pos[1], H_pos[2]])
            H_id = atom_id
            atom_id += 1
            carbon_bonds.append((len(carbon_bonds) + 1, 2, O_id, H_id))

        molecule_id += 1

    bonds = carbon_bonds.copy()
    angles = carbon_angles.copy()
    bond_id = len(bonds) + 1
    angle_id = len(angles) + 1

    # Add Hg2+ ions first (as they were mentioned initially)
    # We'll treat them the same as other cations for simplicity
    added_Hg = []
    for _ in range(num_Hg):
        pos = generate_random_position_box(box_bounds, existing_positions, min_distance=2.5)
        if pos is not None:
            added_Hg.append((4, 2.0, pos))
            existing_positions = np.vstack([existing_positions, pos])

    # Add Mg, Zn, Ca
    added_cations, existing_positions = add_cations(num_Mg, num_Zn, num_Ca, box_bounds, existing_positions)

    # Now sum up total cation charge actually placed
    placed_Hg = len(added_Hg)
    placed_Mg = sum(1 for c in added_cations if c[0] == 6)
    placed_Zn = sum(1 for c in added_cations if c[0] == 7)
    placed_Ca = sum(1 for c in added_cations if c[0] == 8)

    total_cations_charge = placed_Hg*2.0 + placed_Mg*2.0 + placed_Zn*2.0 + placed_Ca*2.0

    required_cl = calculate_required_cl(total_cations_charge)
    # Add cations (Hg, Mg, Zn, Ca) to atoms
    for cat in added_Hg:
        type_id, charge, pos = cat
        atoms.append([atom_id, molecule_id, type_id, charge, pos[0], pos[1], pos[2]])
        atom_id += 1
        molecule_id += 1
    for cat in added_cations:
        type_id, charge, pos = cat
        atoms.append([atom_id, molecule_id, type_id, charge, pos[0], pos[1], pos[2]])
        atom_id += 1
        molecule_id += 1

    # Add Cl- ions
    added_Cl = 0
    for i in range(required_cl):
        pos = generate_random_position_box(box_bounds, existing_positions, min_distance=2.5)
        if pos is not None:
            atoms.append([atom_id, molecule_id, atom_types['Cl'], charges[atom_types['Cl']], pos[0], pos[1], pos[2]])
            atom_id += 1
            molecule_id += 1
            existing_positions = np.vstack([existing_positions, pos])
            added_Cl += 1

    # Finally add water
    num_H2O = 100
    water_molecules, existing_positions = create_water_molecules(num_H2O, box_bounds, existing_positions)
    for water in water_molecules:
        O, H1, H2 = water
        molecule_id += 1
        # O (water)
        atoms.append([atom_id, molecule_id, atom_types['O'], -0.834, O[0], O[1], O[2]])
        O_id = atom_id
        atom_id += 1
        # H1
        atoms.append([atom_id, molecule_id, atom_types['H'], 0.417, H1[0], H1[1], H1[2]])
        H1_id = atom_id
        atom_id += 1
        # H2
        atoms.append([atom_id, molecule_id, atom_types['H'], 0.417, H2[0], H2[1], H2[2]])
        H2_id = atom_id
        atom_id += 1

        bonds.append((bond_id, 2, O_id, H1_id))
        bond_id += 1
        bonds.append((bond_id, 2, O_id, H2_id))
        bond_id += 1
        angles.append((angle_id, 2, H1_id, O_id, H2_id))
        angle_id += 1

    num_atoms = len(atoms)
    num_bonds = len(bonds)
    num_angles = len(angles)

    # Re-check total charge
    total_charge = sum(a[3] for a in atoms)
    if abs(total_charge) > 0.001:
        print(f"Warning: System not neutral. Total charge = {total_charge:.4f}")

    num_bond_types = 3
    num_angle_types = 2

    # Write data file
    with open('data.hg2_sorption', 'w') as f:
        f.write("# LAMMPS data file with corrected charge balance\n\n")
        f.write(f"{num_atoms} atoms\n")
        f.write("8 atom types\n")
        f.write(f"{num_bonds} bonds\n")
        f.write(f"{num_bond_types} bond types\n")
        f.write(f"{num_angles} angles\n")
        f.write(f"{num_angle_types} angle types\n\n")

        f.write(f"{xlo} {xhi} xlo xhi\n")
        f.write(f"{ylo} {yhi} ylo yhi\n")
        f.write(f"{zlo} {zhi} zlo zhi\n\n")

        f.write("Masses\n\n")
        f.write("1 12.011 # C\n")
        f.write("2 15.9994 # O\n")
        f.write("3 1.008 # H\n")
        f.write("4 200.59 # Hg\n")
        f.write("5 35.453 # Cl\n")
        f.write("6 24.305 # Mg\n")
        f.write("7 65.38 # Zn\n")
        f.write("8 40.078 # Ca\n\n")

        f.write("Atoms # full\n\n")
        for atom in atoms:
            f.write(f"{atom[0]} {atom[1]} {atom[2]} {atom[3]:.4f} {atom[4]:.4f} {atom[5]:.4f} {atom[6]:.4f}\n")

        f.write("\nBonds\n\n")
        for bond in bonds:
            f.write(f"{bond[0]} {bond[1]} {bond[2]} {bond[3]}\n")

        f.write("\nAngles\n\n")
        for angle in angles:
            f.write(f"{angle[0]} {angle[1]} {angle[2]} {angle[3]} {angle[4]}\n")

    print(f"LAMMPS data file 'data.hg2_sorption' generated successfully with {num_atoms} atoms.")
    print("Charge neutrality verified.")

    print(f"LAMMPS data file 'data.hg2_sorption' generated successfully with {num_atoms} atoms.")
    print("Charge neutrality verified.")
if __name__ == "__main__":
    main()

LAMMPS data file 'data.hg2_sorption' generated successfully with 1147 atoms.
Charge neutrality verified.
LAMMPS data file 'data.hg2_sorption' generated successfully with 1147 atoms.
Charge neutrality verified.


In [41]:
# Write input script
input_script_content = """
units           real
atom_style      full
boundary        p p p
read_data       data.hg2_sorption

pair_style      lj/cut/coul/long 10.0 10.0
kspace_style    pppm 1.0e-6

# Modify or refine these pair_coeff lines as appropriate for your system
pair_coeff      * * 0.1 3.0

bond_style      harmonic
bond_coeff      1 450.0 1.42
bond_coeff      2 450.0 0.9572
bond_coeff      3 450.0 1.43

angle_style     harmonic
angle_coeff     1 55.0 120.0
angle_coeff     2 55.0 104.52

group           carbon type 1
group           water type 2 3
group           fixed_carbon type 1
fix             fix_carbon fixed_carbon setforce 0.0 0.0 0.0

neighbor        2.0 bin
neigh_modify    delay 0 every 1 check yes

group sorbates subtract all carbon
velocity        sorbates create 300.0 12345 mom yes rot yes dist gaussian
fix             1 water shake 0.0001 20 0 a 1
fix             nvt_control sorbates nvt temp 300.0 300.0 100.0

minimize        1.0e-4 1.0e-6 1000 10000
thermo          1000
thermo_style    custom step temp etotal press
dump            1 all atom 100 dump.hg2_sorption.lammpstrj
run             2000
"""
with open('in.hg2_sorption', 'w') as f:
    f.write(input_script_content)

print(f"LAMMPS data file 'data.hg2_sorption' and input script 'in.hg2_sorption' generated.")
print("Check final total charge and system configuration carefully.")

LAMMPS data file 'data.hg2_sorption' and input script 'in.hg2_sorption' generated.
Check final total charge and system configuration carefully.


In [42]:
# Run the LAMMPS simulation
! /content/lammps/build/lmp -in in.hg2_sorption

LAMMPS (29 Aug 2024 - Update 1)
  using 4 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (0 0 0) to (100 100 35)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  1147 atoms
  scanning bonds ...
  3 = max bonds/atom
  scanning angles ...
  1 = max angles/atom
  orthogonal box = (0 0 0) to (100 100 35)
  1 by 1 by 1 MPI processor grid
  reading bonds ...
  1017 bonds
  reading angles ...
  97 angles
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     4 = max # of 1-2 neighbors
     9 = max # of 1-3 neighbors
    25 = max # of 1-4 neighbors
    20 = max # of special neighbors
  special bonds CPU = 0.001 seconds
  read_data CPU = 0.011 seconds
600 atoms in group carbon
307 atoms in group water
600 atoms in group fixed_carbon
547 atoms in group sorbates
Finding SHAKE clusters ...
       0 = # of size 2 clusters
       0 = # of size 3 clusters
       0 = # of siz